In [19]:
import json

from data_prepare import samples
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import pipeline

from peft import get_peft_model, LoraConfig, TaskType, PeftModel


# Model

In [40]:
model_name = '/Users/kaichen/workspace/data/models/deepseekr1-1.5b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')


# Data

In [41]:
with open('datasets.jsonl', 'w', encoding='utf-8') as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + '\n')
    else:
        print('Data saved to datasets.jsonl')   


Data saved to datasets.jsonl


# Train and test set

In [42]:
dataset = load_dataset("json", data_files={"train": "datasets.jsonl"}, split="train")

Generating train split: 50 examples [00:00, 1283.07 examples/s]


In [43]:
len(dataset)

50

In [44]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [45]:
print(len(train_dataset))
print(len(test_dataset))

45
5


# Tokenizer




该函数的作用是将数据集中的样本转换为模型训练所需的格式，主要包括文本合并、分词处理及标签生成。以下是详细解释：

### 函数功能分解
1. **文本合并**：
   ```python
   text = [f"{prompt}\n{completion}" for prompt, completion in zip(examples['prompt'], examples['completion'])]
   ```
   - 将每个样本的 `prompt`（输入提示）和 `completion`（预期输出）通过换行符 `\n` 拼接成一个完整文本。
   - 例如：若 `prompt` 为“写一首诗”，`completion` 为“春眠不觉晓”，合并后为“写一首诗\n春眠不觉晓”。

2. **分词处理**：
   ```python
   tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512)
   ```
   - 使用预训练的 `tokenizer` 对合并后的文本进行分词。
   - **填充 (Padding)**：将所有序列填充到固定长度（`max_length=512`），确保批量输入时形状统一。
   - **截断 (Truncation)**：若文本超过 512 个 Token，自动截断以适配模型最大长度限制。

3. **生成标签**：
   ```python
   tokens['labels'] = tokens['input_ids'].copy()
   ```
   - 将分词后的 `input_ids`（Token ID 序列）直接复制为 `labels`，用于监督学习。
   - 在自回归语言模型（如 GPT）中，模型的目标是根据上文预测下一个 Token，此时标签应与输入序列一致，但实际训练时需在模型内部将标签右移一位。

### 为什么需要这个函数？
1. **数据格式化**：
   - 模型无法直接处理原始文本，需转换为 Token ID 序列。此函数统一了数据格式，确保所有样本符合模型输入要求。

2. **长度控制**：
   - 通过填充和截断，处理不同长度的文本，避免训练时因序列长度不一致导致的错误。

3. **标签生成**：
   - 为监督学习提供目标标签。在生成任务中，标签帮助模型学习如何从 `prompt` 生成正确的 `completion`。

### 潜在问题与改进
- **标签遮盖**：
  - **问题**：当前函数未区分 `prompt` 和 `completion` 部分的标签，导致模型可能学习预测 `prompt` 本身，而非仅生成 `completion`。
  - **改进**：若需模型仅关注 `completion` 的生成，应将 `prompt` 部分的标签设为 `-100`（损失计算时忽略）。例如：
    ```python
    prompt_tokens = tokenizer(examples['prompt'], add_special_tokens=False)
    prompt_length = len(prompt_tokens['input_ids']) + 1  # +1 为换行符
    labels = [
        [-100] * prompt_length + input_ids[prompt_length:] 
        for input_ids in tokens['input_ids']
    ]
    tokens['labels'] = labels
    ```

### 适用场景
- **预训练任务**：模型需学习完整文本的分布（包括 `prompt` 和 `completion`）。
- **生成任务（需调整）**：若需模型根据 `prompt` 生成 `completion`，需修改标签以屏蔽 `prompt` 部分的损失计算。

### 总结
该函数是数据预处理的核心步骤，负责将原始文本转换为模型可训练的格式，但在实际任务中可能需要根据训练目标调整标签生成逻辑。


In [46]:
def tokenizer_function(examples, tokenizer):
    text = [f"{prompt}\n{completion}" for prompt, completion in zip(examples['prompt'], examples['completion'])]
    tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512)
    tokens['labels'] = tokens['input_ids'].copy()

    return tokens


In [47]:
tokenized_train_dataset = train_dataset.map(lambda examples: tokenizer_function(examples, tokenizer), batched=True)
tokenized_eval_dataset = test_dataset.map(lambda examples: tokenizer_function(examples, tokenizer), batched=True)



Map: 100%|██████████| 5/5 [00:00<00:00, 1545.09 examples/s]


In [48]:
print(tokenized_train_dataset[0])

{'prompt': 'Question 28: Why is it important to practice dynamics in singing?', 'completion': 'Answer 28: Dynamics add emotion and variety to your performance, making your singing more expressive and engaging.', 'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643

# Quantization

GPU required

### `BitsAndBytesConfig` 的详细解释

#### **作用**
`BitsAndBytesConfig` 是 Hugging Face Transformers 库中用于配置 **模型量化（Quantization）** 的类，其核心目标是通过降低模型参数的精度（如将 32 位浮点数转换为 8 位或 4 位整数），显著减少模型的内存占用，使得大型语言模型（如 LLaMA、GPT）能够在资源受限的设备（如消费级 GPU 或 CPU）上运行。

#### **核心功能**
1. **内存优化**：
   - 将模型权重从 `float32` 转换为 `int8` 或 `int4`，内存占用减少至原来的 1/4 或 1/8。
   - 例如：一个 10GB 的模型，8 位量化后仅需约 2.5GB，4 位量化仅需约 1.25GB。

2. **推理加速**：
   - 量化后的模型在支持低精度计算的硬件（如 NVIDIA GPU）上可能更快，但需权衡精度损失。

3. **灵活配置**：
   - 支持混合精度（部分层保留高精度）、阈值控制（跳过小数值的量化）等高级设置。

---

### **参数详解**
以下是 `BitsAndBytesConfig` 的关键参数（以 8 位和 4 位量化为重点）：

| 参数 | 类型 | 作用 | 示例值 |
|------|------|------|--------|
| `load_in_8bit` | `bool` | 启用 8 位量化 | `True` |
| `load_in_4bit` | `bool` | 启用 4 位量化 | `True` |
| `llm_int8_threshold` | `float` | 阈值：超过此值的参数保留为浮点 | `6.0` |
| `bnb_4bit_quant_type` | `str` | 4 位量化的类型（`"fp4"` 或 `"nf4"`） | `"nf4"` |
| `bnb_4bit_compute_dtype` | `torch.dtype` | 4 位量化的计算数据类型 | `torch.float16` |
| `bnb_4bit_use_double_quant` | `bool` | 是否启用双重量化（进一步压缩） | `True` |

---

### **完整用法示例**

#### **1. 8 位量化**
```python
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# 配置 8 位量化，并设置量化阈值
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0  # 绝对值超过 6.0 的参数保留为浮点
)

# 加载量化后的模型
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=quantization_config,
    device_map="auto"  # 自动分配模型层到可用设备（GPU/CPU）
)
```

#### **2. 4 位量化（更高效）**
```python
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",          # 使用 4 位 NormalFloat 量化
    bnb_4bit_compute_dtype=torch.float16,  # 计算时使用 float16 加速
    bnb_4bit_use_double_quant=True      # 启用双重量化（二次压缩）
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
)
```

---

### **使用场景**
| 场景 | 推荐配置 | 说明 |
|------|----------|------|
| 低显存 GPU 推理 | `load_in_8bit=True` | 平衡内存和精度 |
| 超大规模模型部署 | `load_in_4bit=True` + 双重量化 | 极致压缩内存 |
| 高精度需求场景 | `llm_int8_threshold=6.0` | 保留重要参数为浮点 |

---

### **注意事项**
1. **依赖安装**：
   ```bash
   pip install bitsandbytes  # 必须安装量化支持库
   pip install accelerate     # 用于设备自动分配（device_map="auto"）
   ```

2. **性能权衡**：
   - 量化可能导致模型精度下降，尤其在生成任务中可能影响文本连贯性。
   - 4 位量化比 8 位更激进，内存更小，但精度损失更大。

3. **硬件兼容性**：
   - 量化模型在 NVIDIA GPU 上支持最佳，部分操作在 CPU 上可能无法加速。

---

### **完整总结**
`BitsAndBytesConfig` 是 Transformers 库中实现模型量化的核心配置类，通过将模型权重从高精度浮点数转换为低精度整数（8 位或 4 位），显著降低内存占用，使大模型能够在资源受限的环境中运行。其关键功能包括：

1. **内存压缩**：8 位量化减少内存至 1/4，4 位量化至 1/8。
2. **灵活配置**：支持阈值控制、混合精度、双重量化等高级选项。
3. **易用性**：只需在加载模型时传入配置，无需修改模型代码。

**示例总结**：
- **8 位量化**：适合大多数场景，平衡内存和精度。
- **4 位量化**：适合极致内存优化，需搭配 `nf4` 类型和 `float16` 计算以维持性能。

通过合理配置 `BitsAndBytesConfig`，开发者可以在消费级硬件上高效部署百亿参数级别的语言模型。

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# device_map={"cuda:0": "cpu"}
# device_map={"auto"}
model = AutoModelForCausalLM.from_pretrained(model_name, config=quantization_config)
# model.save_pretrained(model_name)




# Lora 

GPU required

In [32]:
lora_config = LoraConfig(
    r=16,  # Number of bits for the mantissa
    lora_alpha=32,  # Scaling factor for low-rank adaptation
    lora_dropout=0.1,  # Dropout rate for low-rank adaptation
    task_type=TaskType.CAUSAL_LM,  # Task type
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# model.save_pretrained(model_name)

trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


# Train parameters setting

In [ ]:
training_args = TrainingArguments(
    output_dir='./fine_tuned_models',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1, # batch size per device during training
    gradient_accumulation_steps=8, # number of updates steps to accumulate before performing a backward/update pass
    fp16=True,                     # GPUWhether to use 16-bit (mixed) precision training instead of 32-bit training.
    eval_steps=10,                 # Number of update steps between two evaluations.
    learning_rate=5e-5,            # learning rate used for training
    logging_dir='./logs',          # directory for storing logs
    logging_steps=10,
    run_name='deepseek-r1-sft-distill',
)

# Trainer

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

ValueError: fp16 mixed precision requires a GPU (not 'mps').

# Train

In [ ]:
trainer.train()
#model.save_pretrained(model_name)
#tokenizer.save_pretrained(model_name)

# Save trained model

## lora model

In [ ]:
save_path = './lora_models'
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

## base model

In [ ]:
final_save_path = './final_models'

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)


# Inference

## load model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)


## inference pipeline

In [ ]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)


In [ ]:
prompt = "What is the meaning of life?"

In [ ]:
generated_text = pipe(prompt, max_length=50, num_return_sequences=1, do_sample=True, temperature=0.8)

In [ ]:
print(generated_text[0]['generated_text'])   